In [1]:
from encoder import tokenizer as t1,m as m_encode
from decoder_2 import tokenizer as t2,m as m_decode

import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import torch
import torch.nn as nn
from torch.nn import functional as F
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time
import inspect
batch_size = 32 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 4
n_layer = 4
dropout = 0.0

1.646988 M parameters
num decayed parameter tensors: 50, with 1,631,744 parameters
num non-decayed parameter tensors: 87, with 15,244 parameters
using fused AdamW: True
Epoch 1 | Loss: 1.336731 | LR: 8.3916e-07 | Grad Norm: 5.7405
Epoch 2 | Loss: 1.323864 | LR: 1.6783e-06 | Grad Norm: 5.8269
Epoch 3 | Loss: 1.306488 | LR: 2.5175e-06 | Grad Norm: 5.6404
Epoch 4 | Loss: 1.273605 | LR: 3.3566e-06 | Grad Norm: 5.5828
Epoch 5 | Loss: 1.242672 | LR: 4.1958e-06 | Grad Norm: 5.3012
Epoch 6 | Loss: 1.193863 | LR: 5.0350e-06 | Grad Norm: 5.1110
Epoch 7 | Loss: 1.153343 | LR: 5.8741e-06 | Grad Norm: 4.6403
Epoch 8 | Loss: 1.112648 | LR: 6.7133e-06 | Grad Norm: 4.0861
Epoch 9 | Loss: 1.097946 | LR: 7.5524e-06 | Grad Norm: 3.4275
Epoch 10 | Loss: 1.036438 | LR: 8.3916e-06 | Grad Norm: 3.3261
Epoch 11 | Loss: 1.032331 | LR: 9.2308e-06 | Grad Norm: 2.7882
Epoch 12 | Loss: 1.016830 | LR: 1.0070e-05 | Grad Norm: 2.5367
Epoch 13 | Loss: 0.996000 | LR: 1.0909e-05 | Grad Norm: 2.3798
Epoch 14 | Loss: 0.96

In [51]:
source_text = "Wait! Who is coming here? "
source_texted=[source_text+" <EOS>"]
seqqed=t1.texts_to_sequences(source_texted)

padsequences = pad_sequences(seqqed, maxlen=64, padding='pre')
source_tokens = torch.tensor(padsequences, dtype=torch.long).to("cuda")  # (1, T)
# Encoder çıkışı (kaynak metnin bağlam temsilini çıkarıyoruz)
with torch.no_grad():
    source_embeddings = m_encode(source_tokens)

In [52]:
target_seq = torch.tensor(t2.texts_to_sequences(["<BOS>"]), dtype=torch.long).to("cuda")  # <BOS> ile başlat
generated = target_seq.clone()
stoi = t2.word_index
itos = dict(zip(stoi.values(), stoi.keys()))
vocab_size = len(stoi)+1

In [53]:
@torch.no_grad()
def generate(model, source_embeddings, max_new_tokens=50, bos_token="<BOS>", eos_token="<EOS>"):
    model.eval()  # Modeli değerlendirme moduna al
    
    # Başlangıç tokenini tensor formatına çevir
    bos_idx = torch.tensor(t2.texts_to_sequences([bos_token])[0], device=device).unsqueeze(0)  # (1, 1)
    
    # Üretilen diziyi başlat
    generated = bos_idx  # (1, 1)
    
    for _ in range(max_new_tokens):
        # Modelden çıktı al
        logits = model(generated, source_embeddings)
          # (1, T, vocab_size)
        logits = logits[:, -1, :]  # Sadece son adımı al (1, vocab_size)
        
        # Olasılıkları softmax ile hesapla
        probs = F.softmax(logits, dim=-1)/0.7  # (1, vocab_size)
       
        # Token seçimi (argmax veya sampling)
        next_token = torch.multinomial(probs, num_samples=1)  # (1, 1)
        
        # Yeni tokeni ekle
        generated = torch.cat([generated, next_token], dim=1)  # (1, T+1)
        
        # Eğer EOS tokenine ulaşıldıysa, döngüyü kır
        if next_token.item() == t2.texts_to_sequences([eos_token])[0][0]:
            break

    return generated  # (1, T')

In [54]:
result=generate(m_decode,source_embeddings)

In [55]:
result

tensor([[  1, 302,  53,  26, 324, 128,   2]], device='cuda:0')

In [56]:
def translate(result):
    eleman=""
    for i in result[0]:
        if(i==1 or i==2):
            continue
        eleman+=" "+itos[int(i)]
    return eleman
translated=translate(result)

In [57]:
print(translated)

 orada patrikler onlara tefeciliği pahalı
